<a href="https://colab.research.google.com/github/jcklpe/neural-network-art/blob/master/neural_style%2Benhance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INTRO**

Hi, this is a script for creating art images using 2 neural networks. One neural network will be used to do transfer the style of one image to another, and the other will be used to expand the image size to a higher resolution. 

You can also copy and paste these commands into the terminal of your local Linux machine (with some modifications) to get the same basic set up. You'll need to install CUDA and CUDNN to your local network first though. 

You can run each of the commands by selecting the code cell and clicking the play button. 

### **SETUP**

#### **Verify NVIDIA card is hooked up to instance.**

In [ ]:
!nvidia-smi

If it's not you need to edit the runtime up in the top menu. 

#### **Import library packages**
This will make it easier to do file system manipulations in python script. We could (and will for some things)run bash subprocesses using the `!` but hey, I thought this would be a fun project to teach myself more python ay!

In [1]:
# for file manipulations
import os

# to color the console out put for better visibility
!pip install colorama
from colorama import Fore, Back, Style

#### **Define project files root**
We've written things this way so that it's more portable. 

In [2]:
projectFiles = "/project-files"
os.makedirs(projectFiles, exist_ok=True)


#### **Mount your google drive to handle your project files**
When you run this it will bring up a link and a form input. Click the link to open an authorization flow to your Google Drive. Then copy the authorization code and paste it into the form. 

You can see the files in the collaboratory instance file system by clicking the folder icon to the left.

In [3]:
from google.colab import drive
drive.mount(f"{projectFiles}/gdrive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /project-files/gdrive


#### **Set up symlink to Google Drive**
The mounted Google Drive contains the folder "My Drive" which causes problems for running the scripts due to the space, so we're going to get around that by symlinking to skip that nasty path.

In [4]:
!ln -sfn '/project-files/gdrive/My Drive/neural-network' {projectFiles}/symdrive

In [5]:
symDrive = f'{projectFiles}/symdrive'

### **Create project variables for project file structure**
Creating these variables abstracts the file structure a little bit. It will make more sense in just a sec. 

In [6]:
os.makedirs(f"{projectFiles}/gdrive/My Drive/neural-network", exist_ok=True)
imageLibrary = f"{symDrive}/image-library"
contentDir = f"{symDrive}/content"
styleDir = f"{symDrive}/style"
outputDir = f"{symDrive}/output"
neuralScript = f"{projectFiles}/neural-style-pt"

#### **Create project folders**
We're going to create a neural-network project folder with 4 sub folders for your project. 

1. The image-library directory holds all the images you plan on using, both content and style images. Lots of images can be used as both so we're going to keep our reference image library in a flat structure. We'll also create a variable for the directories we create for future use.

In [7]:
os.makedirs(imageLibrary, exist_ok=True)

2. This script runs as a batch process so you can process a lot of images and styles automatically. Copy/paste the images you want to run into the content and style folders. These will be cross referenced against each other and exported to the output folder. 

In [8]:
os.makedirs(contentDir, exist_ok=True)
os.makedirs(styleDir, exist_ok=True)

3. And the aformentioned output folder.

In [9]:
os.makedirs(outputDir, exist_ok=True)


#### **Git clone the neural style project, and install the python dependencies**

In [10]:
os.chdir(projectFiles)
!git clone https://github.com/ProGamerGov/neural-style-pt.git 


Cloning into 'neural-style-pt'...
remote: Enumerating objects: 493, done.
remote: Total 493 (delta 0), reused 0 (delta 0), pack-reused 493
Receiving objects: 100% (493/493), 36.23 MiB | 7.10 MiB/s, done.
Resolving deltas: 100% (260/260), done.


#### **Install python dependencies** 

In [11]:
# neural style transfer neural network project
!pip install neural-style

# install torch dep
!pip install torch torchvision

# install pyanime4k for image enhancement
!pip install pyanime4k



     |████████████████████████████████| 13.1MB 243kB/s 


#### **Install the models into the neural-style-pt/models**

In [12]:
!neural-style -download_models /project-files/neural-style-pt/models/

Downloading: "https://web.eecs.umich.edu/~justincj/models/vgg19-d01eb7cb.pth" to /root/.cache/torch/checkpoints/vgg19-d01eb7cb.pth
100% 548M/548M [00:18<00:00, 30.5MB/s]
Downloading: "https://web.eecs.umich.edu/~justincj/models/vgg16-00b39a1b.pth" to /root/.cache/torch/checkpoints/vgg16-00b39a1b.pth
100% 528M/528M [00:18<00:00, 29.4MB/s]
Models have been downloaded to /project-files/neural-style-pt/models/


#### **Define batch process function**

In [23]:
def batchNeural(contentImages, styleImages):
  # move to neural-styles-pt folder context to properly run the command
  os.chdir(f"{projectFiles}/neural-style-pt")

  # iterate through all files in the content folder
  for contentEntry in os.scandir(contentImages):

    # isolate the name of the content image without the extension or path and set to variable.  
    contentFileName = os.path.splitext(contentEntry.name)[0] 

    # make a folder to contain all output of a content kind 
    os.makedirs(f"{outputDir}/{contentFileName}", exist_ok=True) 

    # iterate through all files in the style folder   
    for styleEntry in os.scandir(styleImages):

      # isolate the name of the style image without the extension or path and set to variable.
      styleFileName = os.path.splitext(styleEntry.name)[0]

      #shorten up the output path variables to one variable for readability
      outputFileName = f"{outputDir}/{contentFileName}"

      # run the script!
      !python {neuralScript}/neural_style.py -style_image {styleDir}/{styleEntry.name} -content_image {contentDir}/{contentEntry.name} -save_iter 400 -image_size 600 -num_iterations 1200 -backend cudnn -output_image {outputFileName}/{styleFileName}-{contentFileName}.png
      
      # in order to run pyanime4k we have to run it via python in the
      # command line otherwise it will crash. So we're going to progammatically
      # write a script file and then run it using !python

      importStatement = "import pyanime4k, pathlib\n"
      scriptStatement = f"pyanime4k.upscale_images(input_paths= '{outputFileName}/{styleFileName}-{contentFileName}.png', output_path= pathlib.Path('{outputFileName}'))"

      scriptFile= open("script.py", "w")
      scriptFile.write(f"{importStatement}{scriptStatement}")
      scriptFile.close()

      !python {neuralScript}/script.py
      
      print(Back.BLUE + Fore.YELLOW + Style.BRIGHT + ' =================== ')
      print(f"The '{styleFileName}' style was applied to '{contentFileName}' content image and enhanced")
      print(Back.BLUE + Fore.YELLOW + Style.BRIGHT + ' =================== ' + Style.RESET_ALL)


### **RUN THE SCRIPT!**
Make sure your files are set up in the right folders and invoke the function

In [24]:

batchNeural(contentDir, styleDir)

VGG-19 Architecture Detected
Successfully loaded models/vgg19-d01eb7cb.pth
conv1_1: 64 3 3 3
conv1_2: 64 64 3 3
conv2_1: 128 64 3 3
conv2_2: 128 128 3 3
conv3_1: 256 128 3 3
conv3_2: 256 256 3 3
conv3_3: 256 256 3 3
conv3_4: 256 256 3 3
conv4_1: 512 256 3 3
conv4_2: 512 512 3 3
conv4_3: 512 512 3 3
conv4_4: 512 512 3 3
conv5_1: 512 512 3 3
conv5_2: 512 512 3 3
conv5_3: 512 512 3 3
conv5_4: 512 512 3 3
Setting up style layer 2: relu1_1
Setting up style layer 7: relu2_1
Setting up style layer 12: relu3_1
Setting up style layer 21: relu4_1
Setting up content layer 23: relu4_2
Setting up style layer 30: relu5_1
Capturing content targets
nn.Sequential ( 
  [input -> (1) -> (2) -> (3) -> (4) -> (5) -> (6) -> (7) -> (8) -> (9) -> (10) -> (11) -> (12) -> (13) -> (14) -> (15) -> (16) -> (17) -> (18) -> (19) -> (20) -> (21) -> (22) -> (23) -> (24) -> (25) -> (26) -> (27) -> (28) -> (29) -> (30) -> (31) -> (32) -> (33) -> (34) -> (35) -> (36) -> (37) -> output]
  (1): nn.TVLoss
  (2): nn.Conv2d(3